In [ ]:
import joblib
import pandas as pd
from taxipred.utils.constants import dictionary_for_encoding

model = joblib.load("../models/taxi_rates_XBGRegressor.joblib")

payload = {
    "Time_of_Day": "Morning",
    "Weather": "Clear",
    "Day_of_Week": "Weekday",
    "Traffic_Conditions": "Low",
}
dictionary_for_encoding.append(payload)
# df = pd.DataFrame(dictionary_for_encoding)
# df

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes
0,1.0,Morning,Weekday,1.0,Low,Clear,1.0,10.0,10.0,180.0
1,1.0,Evening,Weekend,1.0,High,Rain,1.0,10.0,10.0,180.0
2,1.0,Afternoon,Weekday,1.0,Medium,Snow,1.0,10.0,10.0,180.0
3,1.0,Night,Weekday,1.0,Low,Clear,1.0,10.0,10.0,180.0
4,NaN,Morning,Weekday,NaN,Low,Clear,NaN,NaN,NaN,NaN


In [68]:
df = pd.DataFrame(dictionary_for_encoding)
df = pd.get_dummies(df, drop_first=True)
df[['Time_of_Day_Evening', 'Time_of_Day_Morning', 'Time_of_Day_Night', 'Day_of_Week_Weekend', 'Traffic_Conditions_Low', 'Traffic_Conditions_Medium', 'Weather_Rain', 'Weather_Snow']].tail(1)
# df]

,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
4,False,True,False,False,True,False,False,False


In [ ]:
from pydantic import BaseModel
from typing import Literal
class TaxiRate(BaseModel): #Detta är "X"
    Time_of_Day: Literal['Morning', 'Evening', 'Afternoon', 'Night'] 
    Weather: Literal['Clear', 'Rain', 'Snow']
    Day_of_Week: Literal['Weekday', 'Weekend'] 
    Traffic_Conditions: Literal['Low', 'High', 'Medium']

validated = []
for data in dictionary_for_encoding:
    validated.append(TaxiRate.model_validate(data))
validated


[TaxiRate(Time_of_Day='Morning', Weather='Clear', Day_of_Week='Weekday', Traffic_Conditions='Low'),
 TaxiRate(Time_of_Day='Evening', Weather='Rain', Day_of_Week='Weekend', Traffic_Conditions='High'),
 TaxiRate(Time_of_Day='Afternoon', Weather='Snow', Day_of_Week='Weekday', Traffic_Conditions='Medium'),
 TaxiRate(Time_of_Day='Night', Weather='Clear', Day_of_Week='Weekday', Traffic_Conditions='Low'),
 TaxiRate(Time_of_Day='Morning', Weather='Clear', Day_of_Week='Weekday', Traffic_Conditions='Low')]

In [ ]:
df_list = []
for data in validated:
    df_list.append(data.model_dump())
df = pd.DataFrame(df_list)
df
validated = pd.get_dummies(df,drop_first=True)
validated
# # dictionary_for_encoding


,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Weather_Rain,Weather_Snow,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium
0,False,True,False,False,False,False,True,False
1,True,False,False,True,False,True,False,False
2,False,False,False,False,True,False,False,True
3,False,False,True,False,False,False,True,False
4,False,True,False,False,False,False,True,False


In [53]:
validated.tail(1)

,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Weather_Rain,Weather_Snow,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium
4,False,True,False,False,False,False,True,False


In [58]:
feature_order = ['Time_of_Day_Evening', 'Time_of_Day_Morning', 'Time_of_Day_Night', 'Day_of_Week_Weekend', 'Traffic_Conditions_Low', 'Traffic_Conditions_Medium', 'Weather_Rain', 'Weather_Snow']

validated = validated[feature_order]
validated

,Time_of_Day_Evening,Time_of_Day_Morning,Time_of_Day_Night,Day_of_Week_Weekend,Traffic_Conditions_Low,Traffic_Conditions_Medium,Weather_Rain,Weather_Snow
0,False,True,False,False,True,False,False,False
1,True,False,False,True,False,False,True,False
2,False,False,False,False,False,True,False,True
3,False,False,True,False,True,False,False,False
4,False,True,False,False,True,False,False,False


In [ ]:
y_pred = model.predict(validated.tail(1))
y_pred

array([[3.474063  , 1.2149789 , 0.29337433]], dtype=float32)

In [71]:
y_pred[0][0]

np.float32(3.474063)